In [16]:
import pandas as pd
fprocessed = "/home/devmiftahul/trading_model/from_finrl-tutorials_git/processed_by_gavin/preprocessed_data_with_features.csv"
processed = pd.read_csv(fprocessed)
processed

,date,tic,close,high,low,open,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,turbulence
0,1993-01-04,ADMG,1.00,1.00,1.00,1.00,1.0,1,0.000000,1.000000,1.000000,100.000000,-66.666667,100.000000,1.000000,1.000000,0.000000
1,1993-01-04,ASII,108.20,110.00,108.20,109.40,5523860.0,1,0.000000,1.000000,1.000000,100.000000,-66.666667,100.000000,108.200000,108.200000,0.000000
2,1993-01-04,BDMN,30350.00,30350.00,30350.00,30350.00,1.0,1,0.000000,1.000000,1.000000,100.000000,-66.666667,100.000000,30350.000000,30350.000000,0.000000
3,1993-01-04,BNGA,6070.00,6070.00,6070.00,6070.00,1.0,1,0.000000,1.000000,1.000000,100.000000,-66.666667,100.000000,6070.000000,6070.000000,0.000000
4,1993-01-04,BNII,2537.27,2537.27,2537.27,2537.27,1.0,1,0.000000,1.000000,1.000000,100.000000,-66.666667,100.000000,2537.270000,2537.270000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
228295,2024-03-08,SMGR,5900.00,5950.00,5900.00,5950.00,6695300.0,7610,-77.012660,6470.763074,5799.236926,41.660961,-145.740281,26.024825,6160.000000,6194.166667,8.296629
228296,2024-03-08,TKIM,6850.00,6950.00,6800.00,6850.00,1895200.0,7610,76.026807,7065.827666,5951.672334,51.211188,137.327368,27.307389,6465.000000,6712.500000,8.296629
228297,2024-03-08,TSPC,1940.00,1960.00,1940.00,1955.00,839100.0,7610,-17.276449,2056.822728,1925.177272,48.258022,-117.575758,10.393875,2014.666667,1964.333333,8.296629
228298,2024-03-08,UNSP,110.00,112.00,105.00,110.00,83600.0,7610,0.606521,115.289621,104.410379,49.705775,11.675265,32.719668,108.366667,110.300000,8.296629


In [2]:
tickers = processed["tic"].unique()
print(len(tickers))
# tickers

30


In [3]:
unique_days = processed["day"].unique()
# len(unique_days)

In [4]:
# # this snippet check what day it is that has the date 2024-09-01 # it is 3570
# x = processed.copy()
# x["date"] = pd.to_datetime(processed["date"])
# target_date = pd.to_datetime("2024-01-02")
# result = x[x["date"] > target_date]
# result

In [5]:
df = processed.copy()
# df_portfolio_train = df[(df["day"] > 3000) & (df["day"] <= 3500)]
df_portfolio_train = df[df["day"] <= 7450]
df_portfolio_test = df[df["day"] > 7450]

In [6]:
# len(df_portfolio_train["tic"].unique())
len(tickers)

30

In [7]:
from finrl.meta.env_portfolio_optimization.env_portfolio_optimization import PortfolioOptimizationEnv

TIME_WINDOW = 63
features=["close", "high", "low"]
initial_features = len(features)

environment_train = PortfolioOptimizationEnv(
        df_portfolio_train,
        initial_amount=100000,
        comission_fee_pct=0.0025,
        time_window=TIME_WINDOW,
        features=features,
        time_column="day",
        normalize_df=None, # dataframe is already normalized
        tics_in_portfolio=tickers
    )
environment_test = PortfolioOptimizationEnv(
        df_portfolio_test,
        initial_amount=100000,
        comission_fee_pct=0.0025,
        time_window=TIME_WINDOW,
        features=features,
        time_column="day",
        normalize_df=None, # dataframe is already normalized
        tics_in_portfolio=tickers
    )

2024-12-02 14:51:14.861260: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-02 14:51:14.882750: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1733125874.906111 1182767 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1733125874.913300 1182767 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-02 14:51:14.942901: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [8]:
# df_portfolio_test
len(df_portfolio_test["day"].unique())

160

In [9]:
import torch
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
print(device)
from finrl.agents.portfolio_optimization.architectures import EIIE
from finrl.agents.portfolio_optimization.models import DRLAgent
model_kwargs = {
    "lr": 0.0001,
    "policy": EIIE,
}
policy_kwargs = {
    "initial_features": initial_features,
    "k_size": 3,
    "conv_mid_features": 2,
    "conv_final_features": 20,
    "time_window": TIME_WINDOW
}

model = DRLAgent(environment_train).get_model("pg", device, model_kwargs, policy_kwargs)

cuda:0


In [10]:
model

In [11]:
DRLAgent.train_model(model, episodes=10)

  0%|                                                                                                                                                    | 0/10 [00:00<?, ?it/s]

Initial portfolio value:100000
Final portfolio value: 8031345049600.0
Final accumulative portfolio value: 80313450.496
Maximum DrawDown: -0.9999731536228875
Sharpe ratio: 0.20180708603191277


 10%|█████████████▉                                                                                                                             | 1/10 [04:46<42:57, 286.34s/it]

Initial portfolio value:100000
Final portfolio value: 2.8873210453208655e+25
Final accumulative portfolio value: 2.8873210453208655e+20
Maximum DrawDown: -0.8565340795034443
Sharpe ratio: 0.43791596488736817


 20%|███████████████████████████▊                                                                                                               | 2/10 [09:36<38:28, 288.51s/it]

...

 80%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████▏                           | 8/10 [38:44<09:42, 291.25s/it]

Initial portfolio value:100000
Final portfolio value: 3.405857522434885e+24
Final accumulative portfolio value: 3.4058575224348852e+19
Maximum DrawDown: -0.9629553915299502
Sharpe ratio: 0.42047454661873457


 90%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████              | 9/10 [43:36<04:51, 291.46s/it]

Initial portfolio value:100000
Final portfolio value: 3.234490728754637e+24
Final accumulative portfolio value: 3.2344907287546372e+19
Maximum DrawDown: -0.9999931573771554
Sharpe ratio: 0.189802859243377


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [48:28<00:00, 290.82s/it]


In [12]:
model

In [13]:
import os
d = "gavin_data_trained_models/eiie"
os.makedirs(d, exist_ok=True)
model_path = f"{d}/policy_EIIE.pt"
torch.save(model.train_policy.state_dict(), model_path)

In [14]:
EIIE_results = {
    "train": environment_train._asset_memory["final"],
    "test": {},
}

# instantiate an architecture with the same arguments used in training
# and load with load_state_dict.
policy = EIIE(**policy_kwargs)
policy.load_state_dict(torch.load(model_path))

# testing
DRLAgent.DRL_validation(model, environment_test, policy=policy)
EIIE_results["test"] = environment_test._asset_memory["final"]

/tmp/ipykernel_1182767/2279313485.py:9: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  policy.load_state_dict(torch.load(model_path))


Initial portfolio value:100000
Final portfolio value: 85750.2265625
Final accumulative portfolio value: 0.857502265625
Maximum DrawDown: -0.23772703125000005
Sharpe ratio: -1.3208057121480483
